# Time series example

*This worksheet was originally designed by [Erin George](https://www.math.ucla.edu/~egeo/) (Department of Mathematics, UCLA). It has been subsequently revised by later TAs and instructors.*

Warning: the model we get is pretty bad!  We used [this dataset](https://www.kaggle.com/felsal/ibovespa-stocks).

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('b3_stocks_1994_2020.csv')

In [3]:
df1 = df.pivot(index='ticker', columns='datetime', values='close')

In [4]:
df2 = df1.loc[:,'2020-12-1':]
df3 = df2.dropna()
df3.head()

datetime,2020-12-10,2020-12-11,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-21,2020-12-22,2020-12-23,2020-12-28,2020-12-29,2020-12-30
ticker,,,,,,,,,,,,,
A1BB34,34.20,33.85,34.18,34.64,34.47,34.63,34.53,35.14,34.99,35.90,36.57,36.14,36.27
A1EG34,18.73,17.89,18.85,18.93,19.11,19.24,18.77,18.67,18.95,19.71,20.35,20.23,20.30
A1KA34,43.83,43.36,44.45,43.92,44.49,43.92,45.10,45.42,45.42,46.28,46.84,46.47,45.32
A1MD34,462.00,462.57,482.00,494.41,494.60,489.87,477.63,474.89,480.46,480.00,481.58,467.90,479.10
A1ZN34,45.19,45.75,42.60,42.75,43.24,42.84,42.75,42.40,41.96,42.49,43.28,43.43,43.59


In [5]:
x = df3.drop(columns=['2020-12-30'])
y = df3['2020-12-30']

In [6]:
from sklearn.model_selection import train_test_split

x_trv, x_test, y_trv, y_test = train_test_split(x, y, random_state=209)
x_train, x_val, y_train, y_val = train_test_split(x_trv, y_trv)

display(x_train.head(1))
x_train.shape

datetime,2020-12-10,2020-12-11,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-21,2020-12-22,2020-12-23,2020-12-28,2020-12-29
ticker,,,,,,,,,,,,
LILY34,410.5,404.81,409.7,425.21,439.22,439.24,433.61,431.69,430.03,433.9,435.7,433.33


(265, 12)

In [7]:
import tensorflow.keras as keras

model = keras.models.Sequential([
    keras.layers.Reshape((x.shape[1], 1), input_shape=(x.shape[1],)),
    keras.layers.Conv1D(10, 10, activation = 'sigmoid'),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 30, activation = 'sigmoid'),
    keras.layers.Dense(units = 1),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 12, 1)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 3, 10)             110       
_________________________________________________________________
flatten (Flatten)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                930       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 1,071
Trainable params: 1,071
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
              loss='mse')

history = model.fit(x_train, 
                    y_train, 
                    epochs=5,
                    validation_data=(x_val, y_val))

Epoch 1/5
9/9 [==============================] - 2s 103ms/step - loss: 45703.3972 - val_loss: 7509.2949
Epoch 2/5
9/9 [==============================] - 0s 9ms/step - loss: 37218.5861 - val_loss: 7464.6846
Epoch 3/5
9/9 [==============================] - 0s 10ms/step - loss: 49849.7371 - val_loss: 7412.3706
Epoch 4/5
9/9 [==============================] - 0s 10ms/step - loss: 27170.1863 - val_loss: 7375.9268
Epoch 5/5
9/9 [==============================] - 0s 9ms/step - loss: 36113.9496 - val_loss: 7342.5854


In [9]:
train_mse = model.evaluate(x_train, y_train)
train_var = y_train.var()

val_mse = model.evaluate(x_val, y_val)
val_var = y_val.var()

print(f'Train RMSE: {np.sqrt(train_mse)}')
print(f'Train r^2: {(train_var - train_mse)/train_var}')

print(f'Validation RMSE: {np.sqrt(val_mse)}')
print(f'Validation r^2: {(val_var - val_mse)/val_var}')

3/3 [==============================] - 0s 2ms/step - loss: 7342.5854
Train RMSE: 197.17772360867747
Train r^2: -0.18459669406880497
Validation RMSE: 85.68888754802894
Validation r^2: -0.3024869180498414
